In [ ]:
import cv2
import torch
import math
import numpy as np
from collections import defaultdict
from IPython.display import display, Image, clear_output
from ultralytics import YOLO

# Function to display images in JupyterLab
def show_image(img):
    _, img_encoded = cv2.imencode('.png', img)
    display(Image(data=img_encoded.tobytes()))

# Euclidean Distance Tracker
class EuclideanDistTracker:
    def __init__(self):
        self.center_points = {}  # id -> (cx, cy, w, h)
        self.id_count = 0
        self.lost_count = {}
        self.timeout = 30

    def update(self, objects_rect):
        objects_bbs_ids = []
        current_ids = set()

        for rect in objects_rect:
            x, y, w, h = rect
            cx = (x + x + w) // 2
            cy = (y + y + h) // 2
            same_object_detected = False

            for id, pt in self.center_points.items():
                dist = math.hypot(cx - pt[0], cy - pt[1])
                size_diff_w = abs(w - pt[2])
                size_diff_h = abs(h - pt[3])

                distance_threshold = max(100, 0.7 * max(w, h))
                size_threshold = max(50, 0.5 * (pt[2] + pt[3]))

                if dist < distance_threshold and size_diff_w < size_threshold and size_diff_h < size_threshold:
                    self.center_points[id] = (cx, cy, w, h)
                    objects_bbs_ids.append([x, y, w, h, id])
                    same_object_detected = True
                    current_ids.add(id)
                    break

            if not same_object_detected:
                self.center_points[self.id_count] = (cx, cy, w, h)
                objects_bbs_ids.append([x, y, w, h, self.id_count])
                self.id_count += 1

        # Handle lost objects
        for id in list(self.center_points.keys()):
            if id not in current_ids:
                self.lost_count[id] = self.lost_count.get(id, 0) + 1
                if self.lost_count[id] > self.timeout:
                    del self.center_points[id]
                    del self.lost_count[id]

        return objects_bbs_ids

# Load YOLOv8 model
model = YOLO('yolov8l.pt')

# Video Capture
cap = cv2.VideoCapture(r'video2.mp4')
tracker = EuclideanDistTracker()

# Video Writer setup
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # Codec
output_video_path = 'output_video.avi'
fps = 30  # Frames per second
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Initialize counters and sets to track unique object IDs
car_count = 0
bike_count = 0
animal_count = 0
unique_car_ids = set()
unique_bike_ids = set()
unique_animal_ids = set()

# Define labels to count
vehicle_labels = ['car', 'motorcycle', 'bicycle']
animal_labels = ['dog', 'cat', 'cow']

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)

    detections = []
    detected_labels = []

    for result in results:
        for detection in result.boxes:
            x1, y1, x2, y2 = detection.xyxy[0].cpu().numpy().astype(int)
            conf = detection.conf[0].item()
            cls = detection.cls[0].item()
            label = model.names[int(cls)]

            if label in vehicle_labels + animal_labels and conf > 0.4:
                detections.append([x1, y1, x2 - x1, y2 - y1])
                detected_labels.append(label)

    # Object tracking
    boxes_ids = tracker.update(detections)

    # Draw detection and tracking results
    for idx, box_id in enumerate(boxes_ids):
        x, y, w, h, object_id = box_id
        label = detected_labels[idx] if idx < len(detected_labels) else "Unknown"
        cv2.putText(frame, f'{label} ID: {object_id}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 3)

        # Update counts and track unique IDs
        if label == 'car' and object_id not in unique_car_ids:
            car_count += 1
            unique_car_ids.add(object_id)
        elif label in ['motorcycle', 'bicycle'] and object_id not in unique_bike_ids:
            bike_count += 1
            unique_bike_ids.add(object_id)
        elif label in animal_labels and object_id not in unique_animal_ids:
            animal_count += 1
            unique_animal_ids.add(object_id)

    # Display counts on the frame
    cv2.putText(frame, f'Cars: {car_count}', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.putText(frame, f'Bikes: {bike_count}', (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.putText(frame, f'Animals: {animal_count}', (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Write the frame to the output video file
    out.write(frame)

    # Display the frame
    clear_output(wait=True)  # Clear previous output
    show_image(frame)  # Display the frame with detections and counts

# Release resources
cap.release()
out.release()  # Release the VideoWriter
cv2.destroyAllWindows()
